In [ ]:
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'ClusterCast.settings'
import test_utils
import django
django.setup()
import sys
sys.path.append("/home/ajp031/StockDeepLearning/ClusterCast/ClusterCast")
from asgiref.sync import sync_to_async
from importlib import reload
import ClusterPipeline.models.ClusterProcessing as cp
import ClusterPipeline.models.SequencePreprocessing as sp
from copy import deepcopy
import numpy as np
from tensorflow.keras.backend import clear_session
import random

In [ ]:
reload(test_utils)

In [73]:
group_ids = [63,66]
groups = await test_utils.extract_cluster_groups(group_ids)

Creating and Processing Dataset
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Empty DataFrame
Columns: []
Index: [2007-01-03 00:00:00]
               macd  macd_signal  macd_diff
Date                                       
2007-01-03 -0.36177     0.002469  -0.441851
Empty DataFrame
Columns: []
Index: [2007-01-03 00:00:00]
              stoch_k    stoch_d
Date                            
2007-01-03  65.766718  60.755066


/usr/remote/apps/anaconda/2023.03-acet116/lib/python3.10/site-packages/pandas/core/internals/concat.py:309: RuntimeWarning: coroutine 'SyncToAsync.__call__' was never awaited
  for blkno, placements in libinternals.get_blkno_placements(blknos, group=False):


Date
2020-08-31    102.095355
2020-09-01    104.520683
2020-09-02    112.636538
2020-09-03     74.485211
2020-09-04     65.968263
2020-09-08     33.111373
2020-09-09     56.779832
2020-09-10     38.381113
2020-09-11     33.402943
2020-09-14     36.965040
2020-09-15     46.871215
2020-09-16     34.274188
2020-09-17     27.100187
2020-09-18     18.362502
2020-09-21     25.278808
2020-09-22     40.783142
2020-09-23     16.850499
2020-09-24     25.923319
2020-09-25     41.793011
2020-09-28     47.250692
Name: bb_indicator60, dtype: float64
Scaling Quant Min Max Features
Quant Min Max Features Scaled
[price_vars, cluster_vars, price_vars, trend_vars, momentum_vars, pctChg_vars, lag_features_vars, pctChgopen_rolling, pctChghigh_rolling, pctChglow_rolling, pctChgclose_rolling, pctChgvolume_rolling, pctChgVix_rolling, pctDiff+sma5_close_rolling, pctDiff+sma5_sma10_rolling, pctDiff+sma5_sma20_rolling, pctDiff+sma5_sma30_rolling, pctDiff+sma5_sma50_rolling, pctDiff+sma5_sma100_rolling, pctDiff+s

In [74]:
clusters = await test_utils.get_all_clusters(groups)

cur_group = groups[0]

In [75]:
cluster_of_interest_id_1 = 600
cluster_of_interest_id_2 = 624
# for cluster in clusters:
#     print(cluster.id)
#     if cluster.id == cluster_of_interest_id_1:
#         cluster_of_interest_1 = cluster
#     if cluster.id == cluster_of_interest_id_2:
#         cluster_of_interest_2 = cluster
cluster_of_interest_1 = [cluster for cluster in clusters if cluster.id == cluster_of_interest_id_1][0]
cluster_of_interest_2 = [cluster for cluster in clusters if cluster.id == cluster_of_interest_id_2][0]

In [76]:
features = groups[0].group_params.training_features

features

['pctChgopen',
 'pctChghigh',
 'pctChglow',
 'pctChgclose',
 'pctChgvolume',
 'pctChgVix',
 'pctDiff+sma5_close',
 'pctDiff+sma5_sma10',
 'pctDiff+sma5_sma20',
 'pctDiff+sma5_sma30',
 'pctDiff+sma5_sma50',
 'pctDiff+sma5_sma100',
 'pctDiff+sma5_sma200',
 'pctDiff+sma10_sma20',
 'pctDiff+sma10_sma30',
 'pctDiff+sma10_sma50',
 'pctDiff+sma10_sma100',
 'pctDiff+sma10_sma200',
 'pctDiff+sma20_sma30',
 'pctDiff+sma20_sma50',
 'pctDiff+sma20_sma100',
 'pctDiff+sma20_sma200',
 'pctDiff+sma30_sma50',
 'pctDiff+sma30_sma100',
 'pctDiff+sma30_sma200',
 'pctDiff+sma50_sma100',
 'pctDiff+sma50_sma200',
 'pctDiff+sma100_sma200',
 'pctDiff+sma10_close',
 'pctDiff+sma20_close',
 'pctDiff+sma30_close',
 'pctDiff+sma50_close',
 'pctDiff+sma100_close',
 'pctDiff+sma200_close',
 'pctDiff+ema5_close',
 'pctDiff+ema10_close',
 'pctDiff+ema20_close',
 'pctDiff+ema30_close',
 'pctDiff+ema50_close',
 'pctDiff+ema100_close',
 'pctDiff+ema200_close',
 'pctDiff+ema5_ema10',
 'pctDiff+ema5_ema20',
 'pctDiff+ema5_

In [77]:
X_train_1, y_train_1, X_test_1, y_test_1 = cluster_of_interest_1.get_3d_array()
X_train_2, y_train_2, X_test_2, y_test_2 = cluster_of_interest_2.get_3d_array()


In [78]:
# best_model_idx = await test_utils.get_best_model(cluster_of_interest_1)
# best_model = cluster_of_interest_1.models[best_model_idx]
best_model = await test_utils.get_model(cluster_of_interest_1, 4136)

In [79]:
reload(test_utils)

<module 'test_utils' from '/home/ajp031/ClusterCast/test_utils.py'>

In [80]:
target_features =groups[0].group_params.target_cols


In [81]:
training_features = best_model.model_features
print(training_features)

['pctDiff+sma10_sma200', 'loCl', 'pctDiff+bb_high_low60', 'pctDiff+ema10_ema100', 'pctDiff+smaVol5_smaVol50', 'pctDiff+ema100_close', 'pctDiff+smaVol20_volume', 'bb_indicator20', 'pctDiff+sma10_sma20', 'pctDiff+ema10_ema30', 'pctDiff+sma100_close', 'deriv+ema20', 'pctChghigh', 'bb_indicator60', 'bb_indicator10', 'pctDiff+sma20_sma50', 'pctDiff+smaVol10_volume', 'deriv+smaVol5', 'deriv+sma10', 'pctChgClHi']


In [82]:
reload(test_utils)

<module 'test_utils' from '/home/ajp031/ClusterCast/test_utils.py'>

In [83]:
print(target_features)
print(len(target_features))

['pctChgclose+1_target', 'pctChgclose+2_target', 'pctChgclose+3_target', 'pctChgclose+4_target', 'pctChgclose+5_target', 'pctChgclose+6_target', 'pctChgclose+7_target', 'pctChgclose+8_target', 'pctChgclose+9_target', 'pctChgclose+10_target', 'pctChgclose+11_target', 'pctChgclose+12_target', 'pctChgclose+13_target', 'pctChgclose+14_target', 'pctChgclose+15_target', 'pctChgclose+16_target', 'pctChgclose+17_target', 'pctChgclose+18_target', 'pctChgclose+19_target', 'pctChgclose+20_target', 'pctChgclose+21_target', 'pctChgclose+22_target', 'pctChgclose+23_target', 'pctChgclose+24_target', 'pctChgclose+25_target']
25


In [84]:
y_feature_dict_1 = groups[0].y_feature_dict
X_feature_dict_1 = groups[0].X_feature_dict

print(X_train_1.shape)
print(y_train_1.shape)
print(X_test_1.shape)
print(y_test_1.shape)
print(y_feature_dict_1)

(396, 75, 517)
(396, 55)
(132, 75, 517)
(132, 55)
{'pctChgclose+10_target': 0, 'pctChgclose+11_target': 1, 'pctChgclose+12_target': 2, 'pctChgclose+13_target': 3, 'pctChgclose+14_target': 4, 'pctChgclose+15_target': 5, 'pctChgclose+16_target': 6, 'pctChgclose+17_target': 7, 'pctChgclose+18_target': 8, 'pctChgclose+19_target': 9, 'pctChgclose+1_target': 10, 'pctChgclose+20_target': 11, 'pctChgclose+21_target': 12, 'pctChgclose+22_target': 13, 'pctChgclose+23_target': 14, 'pctChgclose+24_target': 15, 'pctChgclose+25_target': 16, 'pctChgclose+2_target': 17, 'pctChgclose+3_target': 18, 'pctChgclose+4_target': 19, 'pctChgclose+5_target': 20, 'pctChgclose+6_target': 21, 'pctChgclose+7_target': 22, 'pctChgclose+8_target': 23, 'pctChgclose+9_target': 24, 'pctChgclose-0_target': 25, 'pctChgclose-10_target': 26, 'pctChgclose-11_target': 27, 'pctChgclose-12_target': 28, 'pctChgclose-13_target': 29, 'pctChgclose-14_target': 30, 'pctChgclose-1_target': 31, 'pctChgclose-2_target': 32, 'pctChgclose-3

In [85]:
y_feature_dict_2 = groups[1].y_feature_dict
X_feature_dict_2 = groups[1].X_feature_dict

print(X_train_2.shape)
print(y_train_2.shape)
print(X_test_2.shape)
print(y_test_2.shape)

(686, 75, 517)
(686, 55)
(63, 75, 517)
(63, 55)


In [86]:

X_train_filtered_1 = test_utils.filter_by_features(X_train_1, training_features, X_feature_dict_1)
X_test_filtered_1 = test_utils.filter_by_features(X_test_1, training_features, X_feature_dict_1)
y_train_filtered_1 = test_utils.filter_y_by_features(y_train_1, target_features, y_feature_dict_1)
y_test_filtered_1 = test_utils.filter_y_by_features(y_test_1, target_features, y_feature_dict_1)
print(X_train_filtered_1.shape)
print(y_train_filtered_1.shape)
print(X_test_filtered_1.shape)
print(y_test_filtered_1.shape)

[10, 17, 18, 19, 20, 21, 22, 23, 24, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16]
[10, 17, 18, 19, 20, 21, 22, 23, 24, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16]
(396, 75, 20)
(396, 25)
(132, 75, 20)
(132, 25)


In [87]:
X_train_filtered_2 = test_utils.filter_by_features(X_train_2, training_features, X_feature_dict_2)
X_test_filtered_2 = test_utils.filter_by_features(X_test_2, training_features, X_feature_dict_2)
y_train_filtered_2 = test_utils.filter_y_by_features(y_train_2, target_features, y_feature_dict_2)
y_test_filtered_2 = test_utils.filter_y_by_features(y_test_2, target_features, y_feature_dict_2)
print(X_train_filtered_2.shape)
print(y_train_filtered_2.shape)
print(X_test_filtered_2.shape)
print(y_test_filtered_2.shape)

[10, 17, 18, 19, 20, 21, 22, 23, 24, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16]
[10, 17, 18, 19, 20, 21, 22, 23, 24, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16]
(686, 75, 20)
(686, 25)
(63, 75, 20)
(63, 25)


In [88]:
print(X_train_filtered_1[25, 0,:])

[-0.8102468  -0.32058892 -0.7069009  -1.0290587   0.03380438 -1.1241606
 -0.48551187 -1.2439744  -1.0842304  -1.3581427  -1.0891153  -0.7370113
 -0.44167897 -1.8856703  -0.7770546  -0.9285521  -0.4963388   0.07845228
 -0.5360044  -0.5650144 ]


In [89]:
print(y_train_filtered_1[25, :])

[ 0.4444268  -0.0219647   0.25273412 -0.72043955  0.41170302  0.82556909
  0.09719126 -0.99097878  0.38709009  0.05474026  0.79807091  0.57822233
  0.54846799  0.03119128 -0.66489512  0.15860523 -0.547557   -0.40597922
  0.16198269 -1.          0.25444528 -1.          1.         -0.12147593
 -0.46531907]


In [90]:
best_model.deserialize_model()

2024-03-09 05:09:36.785797: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-09 05:09:36.787986: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-09 05:09:36.789385: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [91]:
og_model = best_model.model

In [92]:
num_days=15
test_utils.eval_model(X_test_filtered_1, y_test_filtered_1, og_model,num_days)

2024-03-09 05:09:41.887759: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-09 05:09:41.890217: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-09 05:09:41.891840: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/5 [=====>........................] - ETA: 8s

2024-03-09 05:09:53.787307: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 9.16MiB (rounded to 9600256)requested by op CudnnRNN
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-03-09 05:09:53.787369: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-03-09 05:09:53.787384: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 45, Chunks in use: 45. 11.2KiB allocated for chunks. 11.2KiB in use in bin. 216B client-requested in use in bin.
2024-03-09 05:09:53.787393: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 300, Chunks in use: 300. 150.2KiB allocated for chunks. 150.2KiB in use in bin. 117.2KiB client-requested in use in bin.
2024-03-09 05:09:53.787400: I tensorflow/tsl/framewor

InternalError: Graph execution error:

Failed to call ThenRnnForward with model config: [rnn_mode, rnn_input_mode, rnn_direction_mode]: 2, 0, 0 , [num_layers, input_size, num_units, dir_count, max_seq_length, batch_size, cell_num_units]: [1, 20, 200, 1, 75, 32, 200] 
	 [[{{node CudnnRNN}}]]
	 [[model_2/encoder_lstm_1_freeze/PartitionedCall]] [Op:__inference_predict_function_23264]

: 

In [ ]:
training_model = test_utils.train_model(training_model, (X_train, y_train, X_test, y_test), training_features, 
                                                   target_features, X_feature_dict, y_feature_dict,
                                                   epochs = 100, batch_size = 12, lr = 0.0005, early_stopping_patience = 15,
                                                   loss = 'mse')

In [ ]:
num_days = 25
attention_model_accuracy, attention_results, predicted_y, attention_weights = test_utils.eval_model(X_test_filtered, y_test_filtered, test_model,num_days)

In [ ]:
print(attention_weights.shape)

In [ ]:
predicted_y_transformed = np.zeros_like(predicted_y)
for i, feature in enumerate(target_features):
    scaler = [feature_set for feature_set in y_feature_sets if feature == feature_set.name][0].scaler
    predicted_y_transformed[:, i] = scaler.inverse_transform(predicted_y[:, i].reshape(-1, 1)).reshape(-1)

In [ ]:
reload(test_utils)
reload(sp)

In [ ]:
print(attention_model_accuracy)

test_utils.visualize_future_distribution(attention_results)

In [ ]:


for i in range(0,20):
    attention_weight = attention_weights[i]
    sequence_element = cluster_of_interest.test_seq_elements[i]
    prediction = predicted_y_transformed[i]
    print(sequence_element.y_feature_sets)
    plot = test_utils.plot_attention_weights(attention_weight)
    plot.show()
    sp.SequenceElement.visualize_future_movement_mpl(sequence_element, prediction, scaler, isCuma = False, target_features=target_features, num_days=num_days)